In [8]:
import pandas as pd
from sklearn import linear_model
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import seaborn as sns

In [42]:
train = pd.read_csv("train_Madison.csv")
#train = train[:20]
test = pd.read_csv("test_Madison.csv")
#names = train["name"]
train_star = train["star"]
train_ID = train['Id']
test_ID = test['Id']
train_name = train['name']
test_name = test['name']
train_city = train['city']
test_city = test['city']
train_post = train['postal_code']
test_post = test['postal_code']
train_text = train['text']
test_text = test['text']

train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)
train.drop('name', axis=1, inplace=True)
test.drop('name', axis=1, inplace=True)
train.drop('city', axis=1, inplace=True)
test.drop('city', axis=1, inplace=True)
train.drop('postal_code', axis=1, inplace=True)
test.drop('postal_code', axis=1, inplace=True)
train.drop('text', axis=1, inplace=True)
test.drop('text', axis=1, inplace=True)


#train.info(verbose=True)
y_train = train_star
#train.head(5)
#test.describe(include = 'all')
city_dummies = pd.get_dummies(train_city)
city_dummies_T = pd.get_dummies(test_city)
#city_dummies.head(5)

count    51313.000000
mean     53685.321653
std         51.222217
min      53508.000000
25%      53703.000000
50%      53703.000000
75%      53711.000000
max      53925.000000
Name: postal_code, dtype: float64

In [25]:
positiveCount = []
negativeCount = []
positiveOverall = []
#ratingArray = []
nwords = []
posnshort = []
negnlong = []
nchars = []
negativeplus = []
posword= []
negword= []
#avgChars = []
posnegratio= []
#numReviews = []
positiveWords = ["knowledgeable", "gem", "highly", "incredible", "amazing", "die", "favorites", "wonderful", "perfect", "fantastic", "notch",
                 "favorite", "awesome", "outstanding", "yum", "delicious", "excellent", "perfectly", "loved", "helpful", "best", 
                 "savory", "cozy", "unique", "yummy", "glad", "homemade", "best", "love", "lovely", "always", "friendly", 
                 "reasonable", "beautiful", "recommended", "fashioned", "classic", "traditional", "great", "fresh", "definitely",
                 "healthy", "decent", "generous", "comfortable", "rich", "recommend", "local", "authentic", "flavorful", "highly",
                 "enjoyed", "pleased", "flavors", "super", "happy", "absolutely", "tasty", "enjoy", "fun", "reasonably", "plenty",
                 "attentive", "truly", "fancy", "wow", "must", "nicely", "every", "quick", "easily", "early", "craving", "well"]
negativeWords = ["disappointing", "weird", "issue", "none", "last", "problem", "dirty", "mediocre", "ok", "poor", "terrible",
                 "awful", "rude", "horrible", "worst", "overpriced", "needed", "barely", "sorry", "waited", "soggy", "waiting",
                 "bland", "forgot", "hope","unfortunately", "sad", "away", "bad", "loud", "overly", "greasy", "frozen", "dry",
                 "empty", "never", "less", "hard", "expensive", "tiny", "however", "wrong", "longer", "nothing", "average", 
                 "cold", "slow", "lack", "avoid", "forget", "left", "worst", "paid", "money", "minutes", "phone","asking",
                 "manager", "paying", "finally", "called", "charge", "customer", "clearly", "bill", "asked","told"]
for i in range(0, len(y_train)):
    #print(i)
    #print(len(names))
    posC = 0
    negC = 0
    for word in positiveWords:
        wordCount = train[word][i]
        posC+=wordCount
    for negWord in negativeWords:
        negWordCount = train[negWord][i]
        negC+=negWordCount
    nword = np.array(train["nword"])[i]
    nchar = np.array(train["nchar"])[i]
    #avgChars.append(nchar/nword)
    nwords.append(nword)
    positiveCount.append(posC)
    negativeCount.append(negC)
    if negC>3:
        negativeplus.append(-1)
    else:
        negativeplus.append(0)
    posnegratio.append((posC)/(negC + 1))
    nchars.append((nchar-604.022230)/532.547462)
    posword.append(posC/nword)
    negword.append(negC*nword)
    if nword<110:
        posnshort.append(posC * 1.5)
        negnlong.append(negC)
    else:
        posnshort.append(posC)
        negnlong.append(negC * 1.5)
    if posC>negC:
        positiveOverall.append(1)
    elif posC<negC:
        positiveOverall.append(-1.5)
    else:
        positiveOverall.append(0)
    #ratingArray.append(ratingAverage[names[i]])
    #numReviews.append(ratingCount[names[i]])
#predictors = np.array([positiveCount, negativeCount, ratingArray])
#predictors = np.array([nwords, positiveOverall, ratingArray, positiveCount, negativeCount, posRatio])
#posRatio = np.array(posRatio)
#print(posRatio)

In [35]:
predictors = np.array([posnegratio, posword, negword, nwords, positiveOverall, posnshort, negnlong, nchars, negativeplus])
predictors = predictors.T
predictors.shape
#x_train = pd.DataFrame(predictors)
#x_train.columns = ['Overall','PosShort','NegLong','nChars', 'negative+']
#x_train.head(5)

(51326, 9)

In [ ]:
x_train = pd.DataFrame(predictors)
x_train.columns = ['Ratio', 'PosShort2','NegLong2','nWords','Overall','PosShort','NegLong','nChars', 'negative+']
#x_train.head(5)
n_folds = 5

def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=96).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return (rmse)

random_forest = RandomForestClassifier()
model_rf = random_forest.fit(x_train, y_train)
reg = linear_model.LinearRegression()
model_reg = reg.fit(x_train, y_train)
knn = KNeighborsClassifier(n_neighbors = 5)
model_knn = knn.fit(x_train, y_train)
linear_svc = LinearSVC()
model_svc = linear_svc.fit(x_train, y_train)
gaussian = GaussianNB()
model_gau = gaussian.fit(x_train, y_train)


In [ ]:

#score1 = rmse_cv(model_rf)
#print("RandomForest score: {:.4f} ({:.4f})\n".format(score1.mean(), score1.std()))
#score2 = rmse_cv(model_reg)
#print("MLR score: {:.4f} ({:.4f})\n".format(score2.mean(), score2.std()))
#score3 = rmse_cv(model_knn)
#print("knn score: {:.4f} ({:.4f})\n".format(score3.mean(), score3.std()))
#score4 = rmse_cv(model_svc)
#print("SVM score: {:.4f} ({:.4f})\n".format(score4.mean(), score4.std()))
#score5 = rmse_cv(model_rf)
#print("NB score: {:.4f} ({:.4f})\n".format(score5.mean(), score5.std()))


In [43]:
acc_RF= round(random_forest.score(x_train, y_train)*100,2)
print(acc_RF)
acc_MLR= round(reg.score(x_train, y_train)*100,2)
print(acc_MLR)
acc_knn= round(knn.score(x_train, y_train)*100,2)
print(acc_knn)
acc_NB= round(gaussian.score(x_train, y_train)*100,2)
print(acc_NB)
acc_linear_svc= round(linear_svc.score(x_train, y_train)*100,2)
print(acc_linear_svc)

75.44
40.02
56.94
42.95
44.76


In [46]:
#testNames = test["name"]
#testIDs = test["Id"]
testExpected = []
for i in range (0, len(test_ID)):
    #name = testNames[i]
    posC = 0
    negC = 0
    posOverall = 0
    for word in positiveWords:
        wordCount = test[word][i]
        posC+=wordCount
    for word in negativeWords:
        wordCount = test[word][i]
        negC+=wordCount 
    if posC > negC:
        posOverall = 1
    elif posC < negC:
        posOverall = -1.5
    else:
        posOverall = 0
    nword = np.array(test["nword"])[i]
    nchar = np.array(test["nchar"])[i]
    avgChar = nchar/nword
    if negC>3:
        negativep = -1
    else:
        negativep = 0
    #numReviews = ratingCount[name]
    standnchars = (nchar-599.985124)/(534.629585)
    if nword<110:
        pns = posC * 1.5
        nnl = negC
    else:
        pns = posC
        nnl = negC * 1.5
    #posnegratio = ((posC)/(negC + 1))
    #predictors = np.array([[posC, negC, rating]])
    predictors = np.array([[posOverall, pns, nnl, standnchars, negativep]])
    prediction = random_forest.predict(predictors)
    #prediction = prediction+rating
    testExpected.append(prediction[0])
    #print(prediction)
submitDF = pd.DataFrame({'Id':test_ID, 'Expected':testExpected})
submitDF.to_csv("submission04.csv", sep=',', index=False)